<a href="https://colab.research.google.com/github/AKI-maggie/thesis/blob/master/ThesisProj2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Thesis Project PART II
This notebook contains the program which re-implemented the algorithms and learning models introduced in the paper of "KE-GAN: Knowledge Embedded Generative Adversarial Networks for Semi-Supervised Scene Parsing".
In this part, the notebook expects to fulfill all implements in the original paper and starts to do all data testings.